In [1]:
import plotly.graph_objects as go
import plotly.express as px
import os
import pandas as pd
import numpy as np
from sqlalchemy import Numeric, Date, create_engine, select, insert
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, Session
import matplotlib.pyplot as plt
from db_tables import *
from utils import *
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
url = os.getenv('PSQL_URL')
engine = create_engine(url, echo=False)

### Method 1: Manual Way

In [36]:

stmt = select(GoldRates)
rows = []
with Session(engine) as session:
    for i, row in enumerate(session.execute(stmt)):
        rows.append(row)

2023-11-13 16:53:03,982 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-11-13 16:53:03,983 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-13 16:53:04,505 INFO sqlalchemy.engine.Engine select current_schema()
2023-11-13 16:53:04,506 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-13 16:53:05,025 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-11-13 16:53:05,026 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-13 16:53:05,453 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-13 16:53:05,455 INFO sqlalchemy.engine.Engine SELECT goldrates.pricedate, goldrates.price22k, goldrates.price24k, goldrates.price18k 
FROM goldrates
2023-11-13 16:53:05,456 INFO sqlalchemy.engine.Engine [generated in 0.00106s] {}
2023-11-13 16:53:06,516 INFO sqlalchemy.engine.Engine ROLLBACK


In [37]:
rows[0][0]

GoldRates(pricedate=2021-07-01, price22k=202.0, price24k=215.0, price18k=165.25)

In [38]:
df_data = [[row[0].pricedate, row[0].price22k, row[0].price24k, row[0].price18k] for row in rows]

In [39]:
df = pd.DataFrame(df_data, columns=['Date', '22k', '24k', '18k'])

In [40]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 865 entries, 2021-07-01 to 2023-11-13
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   22k     865 non-null    object
 1   24k     865 non-null    object
 2   18k     865 non-null    object
dtypes: object(3)
memory usage: 27.0+ KB


### Method 2: Automated

In [9]:
df_2 = pd.read_sql(select(GoldRates), engine, parse_dates=['pricedate'], index_col='pricedate')

In [10]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 865 entries, 2021-07-01 to 2023-11-13
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price22k  865 non-null    float64
 1   price24k  865 non-null    float64
 2   price18k  865 non-null    float64
dtypes: float64(3)
memory usage: 27.0 KB


### Plotly

In [3]:
gold_data = get_gold_rates()

In [32]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=gold_data.index, y=gold_data['price22k'], name='22k'))
fig.add_trace(go.Scatter(x=gold_data.index, y=gold_data['price24k'], name='24k', visible=False))
fig.add_trace(go.Scatter(x=gold_data.index, y=gold_data['price18k'], name='18k', visible=False))
time_buttons = [
    {"count" : 1, 'step' : 'month', 'stepmode' : 'backward', 'label' : '1M'},
    #1 week
    {"count" : 7, 'step' : 'day', 'stepmode' : 'backward', 'label' : '1W'},
    #6 months
    {"count" : 6, 'step' : 'month', 'stepmode' : 'backward', 'label' : '6M'},
]
trace_buttons = [
    {"label" : "22k", "method" : "update", "args" : [{"visible" : [True, False, False]}, {"title" : "22K Gold Rates in AED"}]},
    {"label" : "24k", "method" : "update", "args" : [{"visible" : [False, True, False]}, {"title" : "24K Gold Rates in AED"}]},
    {"label" : "18k", "method" : "update", "args" : [{"visible" : [False, False, True]}, {"title" : "18K Gold Rates in AED"}]},
]

fig.update_layout(
    {
        "xaxis" : {
            "rangeselector" : {
                "buttons" : time_buttons
            },
            #add axis name
            "title" : "Date",
        },
        "yaxis" : {
            "title" : "Price (AED)"
        },
        #add title centred
        "title" : {
            "text" : "Gold Rates in AED",
            "x" : 0.5,
        },
        #add buttons
        "updatemenus" : [
            {
                "type" : "dropdown",
                "direction" : "down",
                "pad" : {"r" : 5, "t" : 10},
                "showactive" : True,
                'active' : 0,
                "x" : 1.1,
                "y" : 0.5,
                "xanchor" : "right",
                "yanchor" : "top",
                "buttons" : trace_buttons,
            }
        ],
    }
)
fig.show()

## Currencies

In [34]:
currencies = pd.read_sql(select(Currencies), engine, parse_dates=['date'], index_col='date')

In [35]:
currencies.head()

,price
date,
2023-05-10,22.30880
2023-05-11,22.35793
2023-05-12,22.40050
2023-05-15,22.40358
2023-05-16,22.41120


In [36]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=currencies.index, y=currencies['price'], name='INR'))
#time buttons
time_buttons = [
    {"count" : 1, 'step' : 'month', 'stepmode' : 'backward', 'label' : '1M'},
    #1 week
    {"count" : 7, 'step' : 'day', 'stepmode' : 'backward', 'label' : '1W'},
    #6 months
    {"count" : 6, 'step' : 'month', 'stepmode' : 'backward', 'label' : '6M'},
]
fig.update_layout(
    {
        "xaxis" : {
            "rangeselector" : {
                "buttons" : time_buttons
            },
            #add axis name
            "title" : "Date",
        },
        "yaxis" : {
            "title" : "Price (AED)"
        },
        #add title centred
        "title" : {
            "text" : "AED to INR",
            "x" : 0.5,
        },
    }
)
fig.show()